# Import libraries

In [1]:
%load_ext autoreload
%autoreload 2

import os
import json

import wandb
import pandas as pd


# Helper Functions

def restore_file(wandb_id, filename='wandb-summary.json', debug=False):
    files_dir = 'notebooks/restored_files'    
    os.system(f'mkdir -p {files_dir}')

    api = wandb.Api()
    if debug:
        run = api.run(f'snne_debug/{wandb_id}')
    else:
        run = api.run(f'snne/{wandb_id}')

    path = f'{files_dir}/{filename}'
    os.system(f'rm -rf {path}')
    run.file(filename).download(root=files_dir, replace=True, exist_ok=False)
    with open(path, 'r') as f:
        out = json.load(f)
    return out


def get_uncertainty_df(metrics):
    data = []
    if 'validation_accuracy' in metrics:
        print(f"Acc: {metrics['validation_accuracy']}")
    else:
        print(f"Acc: {metrics['performance']['accuracy']['mean']}")
    for method in metrics['uncertainty']:
        for metric in metrics['uncertainty'][method]:
            mean = metrics['uncertainty'][method][metric]['mean']
            data.append([method, metric, mean])
    df = pd.DataFrame(data, columns=['method', 'metric', 'means'])
    main_methods = ['semantic_entropy', 'regular_entropy', 'weighted_soft_nearest_neighbor', 'cluster_assignment_entropy', 'p_false', 'soft_nearest_neighbor']
    available_methods = []
    for method in main_methods:
        if method in df.method.to_list():
            available_methods.append(method)
    df = df.set_index('method').loc[available_methods].reset_index()
    main_names = ['SE', 'NE', 'WSNN', 'DSE', 'pTrue', 'SNN']
    conversion = dict(zip(main_methods, main_names))
    df['method'] = df.method.map(lambda x: conversion[x])
    return df

# Extract results

In [3]:
# Fill in the wandb_id assigned to your demo run!

wandb_id = 'fbdqel1s'
if wandb_id == 'YOUR_ID':
    raise ValueError('Need to provide wandb_id of demo run!')

results = restore_file(wandb_id, debug=True)
unc_df = get_uncertainty_df(results)

display(unc_df[unc_df['metric']=='AUROC'])
display(unc_df[unc_df['metric']=='AUARC'])
display(unc_df[unc_df['metric']=='PRR'])

Acc: 0


,method,metric,means
8,SE,AUROC,NaN
12,NE,AUROC,NaN
24,DSE,AUROC,NaN


,method,metric,means
4,SE,AUARC,0.0
11,NE,AUARC,0.0
20,DSE,AUARC,0.0


,method,metric,means
0,SE,PRR,0.5
15,NE,PRR,0.5
21,DSE,PRR,0.5
